In [1]:
import jax
import jax.random as jr 
import jax.numpy as jnp
import haiku as hk
import jraph
import optax

In [ ]:
Graph = jraph.GraphsTuple
ArrayTree = jraph.ArrayTree

In [ ]:
def network_definition(graph: Graph) -> ArrayTree:
    """
    Args:
      graph: GraphsTuple the network processes.

    Returns:
      processed nodes.
    """
    gn = jraph.GraphConvolution(
        update_node_fn=hk.Linear(5, with_bias=False), add_self_edges=False)
    graph = gn(graph)
    graph = graph._replace(nodes=jax.nn.relu(graph.nodes))
    gn = jraph.GraphConvolution(update_node_fn=hk.Linear(1, with_bias=False)) # class variable of nodes?
    graph = gn(graph)
    return graph.nodes

In [ ]:
def train(num_steps: int, run_on_spu: bool):
    """Solves the karte club problem by optimizing the assignments of students."""
    network = hk.without_apply_rng(hk.transform(network_definition))

    params = network.init(jax.random.PRNGKey(42), zacharys_karate_club)

    @jax.jit
    def prediction_loss(params, d, y):
        """ want graph global variable being predicted here... """
        y_ = network.apply(params, d)
        return jnp.square(jnp.subtract(y, y_))

    opt_init, opt_update = optax.adam(1e-3)
    opt_state = opt_init(params)

    @jax.jit
    def update(params, opt_state):
        g = jax.grad(prediction_loss)(params)
        updates, opt_state = opt_update(g, opt_state)
        return optax.apply_updates(params, updates), opt_state

    @jax.jit
    def accuracy(params):
        decoded_nodes = network.apply(params, zacharys_karate_club)

        return jnp.mean(jnp.argmax(decoded_nodes, axis=1) == labels)

    for step in range(num_steps):
        print("step %r accuracy %r", step, accuracy(ppd.get(params)).item())
        print("step %r accuracy %r", step, accuracy(params).item())
        params, opt_state = update(params, opt_state)

    def predict(params):
        decoded_nodes = network.apply(params, zacharys_karate_club)
        return jnp.argmax(decoded_nodes, axis=1)

    print(f"Ground truth: {labels}")
    print(f"CPU prediction: {predict(params)}")
    return accuracy(params).item()